# BashOperator

În cadrul lecției precedente am învățat cum anume să descărcăm anumite date utilizând Python. Datele respective au fost salvate ca și fișier json pe mașina locală. În situații reale, de cele mai multe ori aceste fișier sunt prea mari ca să fie ținute pe o mașină locală. Din acest motiv următorul pas este să ne punem acest fișier în hdfs care este un fișier de date distribuit de la Apache. Ca să facem asta o să ne folosim de operatoul BashOperator

Ca să facem asta trebuie să navigăm întâi pe HUE, sistem prin care putem să facem query la fișier distribuite de tipul HDFS (o să ne salvăm fișierul json creat prin operatorul de Python ca și HDFS). Ca să deschidem HUE o să navigăm la "localhost:32672". Avem HUE instalat pe mașina locală cu ajutorul acelui fișier de docker-compose

<img src="../../ss/airflow-section-03/section-03-ss-07.png">

Aplicația de HUE se vede în imaginea de mai sus. Ce o să facem prin acest operator de bash? O să ne creem un nou director denumit "forex" (directorul o să fie creat în HDFS) și o să ne copiem fișierul "forex_rates.json" în acest director nou creat. Comenzile de bash pentru a face asta sunt următoarele:

In [ ]:
hdfs dfs -mkdir -p /forex
hdfs dfs -put -f $AIRFLOW_HOME/dags/files/forex_rates.json /forex

Aceste comenzi trebuie să le introducem în cadrul operatorului de Bash. Prin operatorul respectiv, oferindu-i aceste comenzi o să ne copiem fișierul respectiv în HDFS. Codul pentru a face asta este următorul:

In [ ]:
from airflow.operators.bash import BashOperator

save_forex_rates_to_hdfs = BashOperator(
    task_id="save_forex_rates_to_hdfs",
    bash_command="""
        hdfs dfs -mkdir -p /forex && \
        hdfs dfs -put -f $AIRFLOW_HOME/dags/files/forex_rates.json /forex
    """
)

Codul cu tot DAG-ul este următrorul:

In [ ]:
from airflow import DAG
from airflow.providers.http.sensors.http import HttpSensor
from airflow.sensors.filesystem import FileSensor
from airflow.operators.python import PythonOperator
from airflow.operators.bash import BashOperator

import json
import csv
import requests
from datetime import datetime, timedelta


# Define the default_args
default_args = {
    "owner": "airflow",
    "retries": 3,
    "retry_delay": timedelta(minutes=5),
    "email_on_failure": False,
    "email": "georgiuandrei05@gmail.com"
}

# Define the function that donwloads the data from the API
def download_rates():
    BASE_URL = "https://gist.githubusercontent.com/marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b/raw/"
    ENDPOINTS = {
        'USD': 'api_forex_exchange_usd.json',
        'EUR': 'api_forex_exchange_eur.json'
    }
    with open('/opt/airflow/dags/files/forex_currencies.csv') as forex_currencies:
        reader = csv.DictReader(forex_currencies, delimiter=';')
        for idx, row in enumerate(reader):
            base = row['base']
            with_pairs = row['with_pairs'].split(' ')
            indata = requests.get(f"{BASE_URL}{ENDPOINTS[base]}").json()
            outdata = {'base': base, 'rates': {}, 'last_update': indata['date']}
            for pair in with_pairs:
                outdata['rates'][pair] = indata['rates'][pair]
            with open('/opt/airflow/dags/files/forex_rates.json', 'a') as outfile:
                json.dump(outdata, outfile)
                outfile.write('\n')

# Define the DAG
with DAG(
    dag_id = 'test_forex_data_pipeline',
    description='Testing the Data Forex Pipeline from Udemy Course',
    schedule_interval="@daily",
    start_date=datetime(2024, 1, 1),
    catchup=False
    ) as dag:
    
    # Define the HttpSensor task
    is_forex_rates_available = HttpSensor(
        task_id="is_forex_rates_available",
        http_conn_id="forex_api",
        endpoint="marclamberti/f45f872dea4dfd3eaa015a4a1af4b39b",
        response_check=lambda response: "rates" in response.text,
        poke_interval=5,
        timeout=25
    )

    # Define the FileSensor task
    is_forex_file_available = FileSensor(
        task_id="is_forex_file_available",
        fs_conn_id="forex_path",
        filepath="forex_currencies.csv",
        poke_interval=5,
        timeout=20
    )

    # Define the PythonOperator task
    download_forex_rates = PythonOperator(
        task_id="download_forex_rates",
        python_callable=download_rates
    )

    # Define the BashOperator
    save_forex_rates_to_hdfs = BashOperator(
        task_id="save_forex_rates_to_hdfs",
        bash_command="""
            hdfs dfs -mkdir -p /forex && \
            hdfs dfs -put -f $AIRFLOW_HOME/dags/files/forex_rates.json /forex
        """
    )

